# Denoise
We may want to do some basic denoising to clean up the image

In [3]:
from skimage.restoration import (denoise_tv_chambolle, denoise_bilateral,
                                 denoise_wavelet, estimate_sigma)
from astropy.io import fits

In [48]:
mosaic_hdu = fits.open('M101.fits')[0]

In [5]:
denoised = denoise_tv_chambolle(mosaic_hdu.data, weight=0.1, multichannel=True)
mosaic_hdu.data = denoised
mosaic_hdu.writeto('denoised.fits', overwrite=True)

In [36]:
from PIL import Image, ImageFilter, ImageEnhance
import matplotlib.pyplot as plt
import numpy as np

In [52]:
mosaic_hdu = fits.open('M101.fits')[0]
image = Image.fromarray(mosaic_hdu.data.astype('uint8'))
new_image = image.filter(ImageFilter.UnsharpMask(radius=2, percent=500))
mosaic_hdu.data = new_image
mosaic_hdu.writeto('denoised_unsharp.fits', overwrite=True)

In [50]:
mosaic_hdu = fits.open('M101.fits')[0]
image = Image.fromarray(mosaic_hdu.data.astype('uint8'))
new_image = image.filter(ImageFilter.BoxBlur(radius=1))
mosaic_hdu.data = new_image
mosaic_hdu.writeto('denoised_blur.fits', overwrite=True)

In [53]:
mosaic_hdu = fits.open('M101.fits')[0]
new_image = image.filter(ImageFilter.MedianFilter())
mosaic_hdu.data = new_image
mosaic_hdu.writeto('denoised_enh.fits', overwrite=True)

In [54]:
mosaic_hdu = fits.open('M101.fits')[0]
new_image = image.filter(ImageFilter.ModeFilter())
mosaic_hdu.data = new_image
mosaic_hdu.writeto('denoised_mode.fits', overwrite=True)

In [55]:
from scipy import fftpack


In [75]:
mosaic_hdu = fits.open('stacked_1.fits')[0]
image = Image.fromarray(mosaic_hdu.data.astype('uint8'))
im_fft = fftpack.fft2(image)

keep_fraction = 0.2

# Call ff a copy of the original transform. Numpy arrays have a copy
# method for this purpose.
im_fft2 = im_fft.copy()

# Set r and c to be the number of rows and columns of the array.
r, c = im_fft2.shape

# Set to zero all rows with indices between r*keep_fraction and
# r*(1-keep_fraction):
im_fft2[int(r*keep_fraction):int(r*(1-keep_fraction))] = 0

# Similarly with the columns:
im_fft2[:, int(c*keep_fraction):int(c*(1-keep_fraction))] = 0

im_new = fftpack.ifft2(im_fft2).real
mosaic_hdu.data = im_new
mosaic_hdu.writeto('denoised_fft.fits', overwrite=True)


In [73]:
mosaic_hdu = fits.open('M101.fits')[0]
image = Image.fromarray(mosaic_hdu.data.astype('uint8'))
mosaic_hdu.data = (image - im_new)
mosaic_hdu.writeto('denoised_fft_diff.fits', overwrite=True)